In [ ]:
import numpy as np
from src.perceptron import Perceptron
from src.trainer import TrainerConfig, train_perceptron, evaluate_perceptron
from src.readers import read_csv

In [ ]:
dataset, dataset_outputs = read_csv('TP3-ej2-conjunto.csv')

config = TrainerConfig.from_file("ejercicio2-lineal-config.json")

In [ ]:
perceptron_lin = Perceptron(
    initial_weights=np.random.random(len(dataset[0]) + 1) * 2 - 1,
    theta_func=config.theta
)

print(f"Initial weights for LINEAR perceptron: {perceptron_lin.w}")

error = evaluate_perceptron(
    perceptron=perceptron_lin,
    dataset=dataset,
    dataset_outputs=dataset_outputs,
    error_func=config.error_func,
    print_output=True,
    acceptable_error=config.acceptable_error
)

print(f"LINEAR perceptron initially has an error of {error} {'✅' if error<=config.acceptable_error else '❌'}")

In [ ]:
result_lin = train_perceptron(
    perceptron=perceptron_lin,
    dataset=dataset,
    dataset_outputs=dataset_outputs,
    config=config
)

In [ ]:
error = evaluate_perceptron(
    perceptron=perceptron_lin,
    dataset=dataset,
    dataset_outputs=dataset_outputs,
    error_func=config.error_func,
    print_output=True,
    acceptable_error=config.acceptable_error
)

print(f"LINEAR perceptron after training for {result_lin.epoch_num} epoch{'' if result_lin.epoch_num == 1 else 's'} has an error of {error} {'✅' if error<=config.acceptable_error else '❌'}")
print(f"Finished because: {result_lin.end_reason.value}")
print(f"Final weights: {perceptron_lin.w}")

In [ ]:
config = TrainerConfig.from_file("ejercicio2-tanh-config.json")

perceptron_tanh = Perceptron(
    initial_weights=np.random.random(len(dataset[0]) + 1) * 2 - 1,
    theta_func=config.theta
)
outputs_tanh = config.scaler.scale(dataset_outputs)

print(f"Initial weights for TANH perceptron: {perceptron_tanh.w}")

error = evaluate_perceptron(
    perceptron=perceptron_tanh,
    dataset=dataset,
    dataset_outputs=outputs_tanh,
    error_func=config.error_func,
    print_output=True,
    acceptable_error=config.acceptable_error
)

print(f"TANH perceptron initially has an error of {error} {'✅' if error<=config.acceptable_error else '❌'}")

In [ ]:
result_tanh = train_perceptron(
    perceptron=perceptron_tanh,
    dataset=dataset,
    dataset_outputs=outputs_tanh,
    config=config
)

In [ ]:
error = evaluate_perceptron(
    perceptron=perceptron_tanh,
    dataset=dataset,
    dataset_outputs=outputs_tanh,
    error_func=config.error_func,
    print_output=True,
    acceptable_error=config.acceptable_error
)

print(f"TANH perceptron after training for {result_tanh.epoch_num} epoch{'' if result_tanh.epoch_num == 1 else 's'} has an error of {error} {'✅' if error<=config.acceptable_error else '❌'}")
print(f"Finished because: {result_tanh.end_reason.value}")
print(f"Final weights: {perceptron_tanh.w}")

In [ ]:
config = TrainerConfig.from_file("ejercicio2-logistic-config.json")

perceptron_logis = Perceptron(
    initial_weights=np.random.random(len(dataset[0]) + 1) * 2 - 1,
    theta_func=config.theta
)
outputs_logis = config.scaler.scale(dataset_outputs)

print(f"Initial weights for LOGISTIC perceptron: {perceptron_logis.w}")

error = evaluate_perceptron(
    perceptron=perceptron_logis,
    dataset=dataset,
    dataset_outputs=outputs_logis,
    error_func=config.error_func,
    print_output=True,
    acceptable_error=config.acceptable_error
)

print(f"LOGISTIC perceptron initially has an error of {error} {'✅' if error<=config.acceptable_error else '❌'}")

In [ ]:
result_logis = train_perceptron(
    perceptron=perceptron_logis,
    dataset=dataset,
    dataset_outputs=outputs_logis,
    config=config
)

In [ ]:
error = evaluate_perceptron(
    perceptron=perceptron_logis,
    dataset=dataset,
    dataset_outputs=outputs_logis,
    error_func=config.error_func,
    print_output=True,
    acceptable_error=config.acceptable_error
)

print(f"LOGISTIC perceptron after training for {result_logis.epoch_num} epoch{'' if result_logis.epoch_num == 1 else 's'} has an error of {error} {'✅' if error<=config.acceptable_error else '❌'}")
print(f"Finished because: {result_logis.end_reason.value}")
print(f"Final weights: {perceptron_logis.w}")